# 1. Voting Classifier
#### In this assignment, you are expected to build an ensemble of different models and train it on cover type dataset.

## 1.1. Load dataset
#### You will need to read the data from the file (cover.csv). It contains 581012 samples and 54 attributes for each sample. The target column is Cover_Type.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.linear_model import LinearRegression, LogisticRegression,  Ridge, SGDClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV,  cross_val_score,GridSearchCV, ShuffleSplit

from sklearn.preprocessing import MinMaxScaler
import math
import xgboost as xgb
from sklearn.ensemble import VotingClassifier, RandomForestRegressor, RandomForestClassifier, ExtraTreesClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report
import xgboost
from sklearn.svm import SVC, LinearSVC
from sklearn.neural_network import MLPClassifier

from scipy.stats import mode

from sklearn.tree import DecisionTreeClassifier

In [2]:
# Loading data
df=pd.read_csv('cover.csv')

In [3]:
df

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,Cover_Type
0,2596.0,51.0,3.0,258.0,0.0,510.0,221.0,232.0,148.0,6279.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
1,2590.0,56.0,2.0,212.0,-6.0,390.0,220.0,235.0,151.0,6225.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
2,2804.0,139.0,9.0,268.0,65.0,3180.0,234.0,238.0,135.0,6121.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,2785.0,155.0,18.0,242.0,118.0,3090.0,238.0,238.0,122.0,6211.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
4,2595.0,45.0,2.0,153.0,-1.0,391.0,220.0,234.0,150.0,6172.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396.0,153.0,20.0,85.0,17.0,108.0,240.0,237.0,118.0,837.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
581008,2391.0,152.0,19.0,67.0,12.0,95.0,240.0,237.0,119.0,845.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
581009,2386.0,159.0,17.0,60.0,7.0,90.0,236.0,241.0,130.0,854.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
581010,2384.0,170.0,15.0,60.0,5.0,90.0,230.0,245.0,143.0,864.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


In [4]:
# Getting info about data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581012 entries, 0 to 581011
Data columns (total 55 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   Elevation                           581012 non-null  float64
 1   Aspect                              581012 non-null  float64
 2   Slope                               581012 non-null  float64
 3   Horizontal_Distance_To_Hydrology    581012 non-null  float64
 4   Vertical_Distance_To_Hydrology      581012 non-null  float64
 5   Horizontal_Distance_To_Roadways     581012 non-null  float64
 6   Hillshade_9am                       581012 non-null  float64
 7   Hillshade_Noon                      581012 non-null  float64
 8   Hillshade_3pm                       581012 non-null  float64
 9   Horizontal_Distance_To_Fire_Points  581012 non-null  float64
 10  Wilderness_Area_0                   581012 non-null  float64
 11  Wilderness_Area_1         

In [5]:
# Checking values if there is null value or not.
df.isnull().sum().sum()

0

In [ ]:
df.describe()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,Cover_Type
count,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,...,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000
mean,2959.365301,155.656807,14.103704,269.428217,46.418855,2350.146611,212.146049,223.318716,142.528263,1980.291226,...,0.090392,0.077716,0.002773,0.003255,0.000205,0.000513,0.026803,0.023762,0.015060,2.051471
std,279.984734,111.913721,7.488242,212.549356,58.295232,1559.254870,26.769889,19.768697,38.274529,1324.195210,...,0.286743,0.267725,0.052584,0.056957,0.014310,0.022641,0.161508,0.152307,0.121791,1.396504
min,1859.000000,0.000000,0.000000,0.000000,-173.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2809.000000,58.000000,9.000000,108.000000,7.000000,1106.000000,198.000000,213.000000,119.000000,1024.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,2996.000000,127.000000,13.000000,218.000000,30.000000,1997.000000,218.000000,226.000000,143.000000,1710.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
75%,3163.000000,260.000000,18.000000,384.000000,69.000000,3328.000000,231.000000,237.000000,168.000000,2550.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
max,3858.000000,360.000000,66.000000,1397.000000,601.000000,7117.000000,254.000000,254.000000,254.000000,7173.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000


In [6]:
# Initializing data
X=df.drop(['Cover_Type'],axis=1)
y=df['Cover_Type']

In [7]:
# Checking:
X.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_30,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39
0,2596.0,51.0,3.0,258.0,0.0,510.0,221.0,232.0,148.0,6279.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2590.0,56.0,2.0,212.0,-6.0,390.0,220.0,235.0,151.0,6225.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2804.0,139.0,9.0,268.0,65.0,3180.0,234.0,238.0,135.0,6121.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2785.0,155.0,18.0,242.0,118.0,3090.0,238.0,238.0,122.0,6211.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2595.0,45.0,2.0,153.0,-1.0,391.0,220.0,234.0,150.0,6172.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
y.unique()

array([5, 2, 1, 7, 3, 6, 4])

## 1.2. Prepare dataset
#### Split the data into train, validation, and test sets using train_test_split twice with 0.2 test_size. Your final distribution will be 371847-92962-116203.

In [8]:
# Splitting the data into train, validation, and test sets using train_test_split twice with 0.2 test_size.
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=14)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=14)

In [9]:
# Checking final distribution:
print("Training data: ",X_train.shape[0],y_train.shape[0])
print("Validation data: ",X_val.shape[0],y_val.shape[0])
print("Test data: ",X_test.shape[0],y_test.shape[0])

Training data:  371847 371847
Validation data:  92962 92962
Test data:  116203 116203


## 1.3. Modeling
#### Train 4-5 different classifiers on the data. You can train RandomForestClassifier, ExtraTreesClassifier, LinearSVC, SGDClassifier, MLPClassifier, etc. Evaluate their performances using validation set. Note that training may take quite a while (up to 30 minutes) depending on the hardware.

In [ ]:
# List of classifiers
classifiers = {
    "RandomForestClassifier": RandomForestClassifier(),
    "ExtraTreesClassifier": ExtraTreesClassifier(),
    "LinearSVC": LinearSVC(),
    "SGDClassifier": SGDClassifier(max_iter=1000),
    "MLPClassifier": MLPClassifier()
}

# Train and evaluate each classifier
results = {}
for name, clf in classifiers.items():
    print(f"Training {name}...")
    clf.fit(X_train, y_train)  # Train the classifier
    y_pred = clf.predict(X_val)  # Predict on validation set

    # Evaluate performance
    accuracy = accuracy_score(y_val, y_pred )
    print(f"{name} Accuracy: {accuracy:.5f}")
    print(f"Classification Report for {name}:\n{classification_report(y_val, y_pred, zero_division=0)}\n")

    # Save results
    results[name] = accuracy

# Display summary of results
print("Summary of Classifier Performance:")
for name, acc in results.items():
    print(f"{name}: {acc:.5f}")

Training RandomForestClassifier...
RandomForestClassifier Accuracy: 0.94953
Classification Report for RandomForestClassifier:
              precision    recall  f1-score   support

           1       0.96      0.94      0.95     33916
           2       0.95      0.97      0.96     45320
           3       0.93      0.95      0.94      5712
           4       0.91      0.82      0.86       452
           5       0.94      0.76      0.84      1497
           6       0.91      0.88      0.90      2716
           7       0.97      0.94      0.96      3349

    accuracy                           0.95     92962
   macro avg       0.94      0.90      0.92     92962
weighted avg       0.95      0.95      0.95     92962


Training ExtraTreesClassifier...
ExtraTreesClassifier Accuracy: 0.94856
Classification Report for ExtraTreesClassifier:
              precision    recall  f1-score   support

           1       0.96      0.93      0.95     33916
           2       0.94      0.97      0.96    

## 1.4. Ensembling
#### Create a hard and soft voting classifier using the models you have trained. You can use VotingClassifier. Check its performance on the validation set. Do you get better or worse performance than any of the individual classifiers?

In [10]:
# VotingClassifier
clf1=RandomForestClassifier()
clf2=ExtraTreesClassifier()
clf3=LinearSVC()
clf4=SGDClassifier(max_iter=1000)
clf5=MLPClassifier()


In [8]:


hardvoting_clf=VotingClassifier(estimators=[('rf',clf1),('etc',clf2),('lsvc',clf3),('sgd',clf4),('mlp',clf5)],voting='hard')
hardvoting_clf.fit(X_train,y_train)
y_hardvoting_predict=hardvoting_clf.predict(X_val) # Checking performance



D:\Ehtiram\Documents\Anaconda\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
D:\Ehtiram\Documents\Anaconda\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
D:\Ehtiram\Documents\Anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
df

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,Cover_Type
0,2596.0,51.0,3.0,258.0,0.0,510.0,221.0,232.0,148.0,6279.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
1,2590.0,56.0,2.0,212.0,-6.0,390.0,220.0,235.0,151.0,6225.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
2,2804.0,139.0,9.0,268.0,65.0,3180.0,234.0,238.0,135.0,6121.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,2785.0,155.0,18.0,242.0,118.0,3090.0,238.0,238.0,122.0,6211.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
4,2595.0,45.0,2.0,153.0,-1.0,391.0,220.0,234.0,150.0,6172.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396.0,153.0,20.0,85.0,17.0,108.0,240.0,237.0,118.0,837.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
581008,2391.0,152.0,19.0,67.0,12.0,95.0,240.0,237.0,119.0,845.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
581009,2386.0,159.0,17.0,60.0,7.0,90.0,236.0,241.0,130.0,854.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
581010,2384.0,170.0,15.0,60.0,5.0,90.0,230.0,245.0,143.0,864.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


In [ ]:
# Evaluate performance
accuracy_hardvoting = accuracy_score(y_val, y_hardvoting_predict)
print("Accuracy Hard VotingClassifier: ",accuracy_hardvoting)
print("Classification Report for Hard VotingClassifier: \n", classification_report(y_val, y_hardvoting_predict))

Accuracy Hard VotingClassifier:  0.8591144768830274
Classification Report for Hard VotingClassifier: 
               precision    recall  f1-score   support

           1       0.94      0.75      0.84     33916
           2       0.81      0.98      0.88     45320
           3       0.86      0.92      0.89      5712
           4       0.93      0.74      0.82       452
           5       0.97      0.18      0.30      1497
           6       0.94      0.58      0.72      2716
           7       0.98      0.78      0.87      3349

    accuracy                           0.86     92962
   macro avg       0.92      0.70      0.76     92962
weighted avg       0.87      0.86      0.85     92962



In [11]:
# VotingClassifier
clf1=RandomForestClassifier()
clf2=ExtraTreesClassifier()
clf3=LinearSVC()  #does  not support predict_proba
clf4=SGDClassifier(max_iter=1000) #does not support predict_proba
clf5=MLPClassifier()

In [ ]:
softvoting_clf=VotingClassifier(estimators=[('rf',clf1),('etc',clf2),('mlp',clf5)],voting='soft')
softvoting_clf.fit(X_train,y_train)
y_softvoting_predict=softvoting_clf.predict(X_val) # Checking performance

In [ ]:
# Evaluate performance
accuracy_softvoting = accuracy_score(y_val, y_softvoting_predict)
print("Accuracy Soft VotingClassifier: ",accuracy_softvoting)
print("Classification Report for Soft VotingClassifier: \n", classification_report(y_val, y_softvoting_predict))

Accuracy Soft VotingClassifier:  0.9318861470278178
Classification Report for Soft VotingClassifier: 
               precision    recall  f1-score   support

           1       0.95      0.91      0.93     33916
           2       0.92      0.96      0.94     45320
           3       0.91      0.95      0.93      5712
           4       0.89      0.84      0.87       452
           5       0.94      0.68      0.79      1497
           6       0.91      0.83      0.87      2716
           7       0.98      0.91      0.94      3349

    accuracy                           0.93     92962
   macro avg       0.93      0.87      0.89     92962
weighted avg       0.93      0.93      0.93     92962



#### Check if any of the models hurts the performance of the ensemble. You can access the estimators of the ensemble using estimators_ attribute. If so, drop those using set_params and reevaluate.

---



---



In [17]:
print(hardvoting_clf.estimators_)

[RandomForestClassifier(), ExtraTreesClassifier(), LinearSVC(), SGDClassifier(), MLPClassifier()]


In [12]:


# VotingClassifier with all classifiers
hardvoting_clf = VotingClassifier(
    estimators=[('rf', clf1), ('etc', clf2), ('lsvc', clf3), ('sgd', clf4), ('mlp', clf5)], 
    voting='hard'
)

# Keep only RandomForest and ExtraTrees by setting other estimators to 'drop' because only they are improving overall model
# performance as we see previously with their individiual accuraacy
hardvoting_clf.set_params(
    lsvc='drop', 
    sgd='drop', 
    mlp='drop'
)

# Refit the VotingClassifier with only RandomForest and ExtraTrees
hardvoting_clf.fit(X_train, y_train)

# Make predictions and check performance
y_hardvoting_predict = hardvoting_clf.predict(X_val)
performance = accuracy_score(y_val, y_hardvoting_predict)
print(f"Performance with RandomForest and ExtraTrees: {performance}")


Performance with RandomForest and ExtraTrees: 0.9499150190400378


# 2. Random Forest
#### In this assignment, you are expected to build a random forest that classifies a toy dataset.

## 2.1. Load dataset
#### You will need to read the data from the file (data.csv). It contains 15000 samples and two features for each sample.

In [2]:
df=pd.read_csv('data.csv',header=None)
df

,0,1,2
0,10182.554999,-371.830691,100.0
1,-8493.323486,7009.446179,0.0
2,21322.088204,-390.558362,100.0
3,5473.925002,-1878.223941,0.0
4,-7422.540710,5291.351276,0.0
...,...,...,...
14995,-7273.210307,5827.740028,0.0
14996,16408.532323,1111.727000,100.0
14997,-7732.465058,14232.038118,0.0
14998,6249.812673,6541.827106,0.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       15000 non-null  float64
 1   1       15000 non-null  float64
 2   2       15000 non-null  float64
dtypes: float64(3)
memory usage: 351.7 KB


In [4]:
df.describe()

,0,1,2
count,15000.000000,15000.000000,15000.000000
mean,4985.458275,2475.636837,50.000000
std,9549.756718,6315.960957,50.001667
min,-22796.187831,-17097.467060,0.000000
25%,-1870.390611,-2159.961278,0.000000
50%,5025.481917,2406.963192,50.000000
75%,11882.722036,7095.710281,100.000000
max,33426.402909,23575.683892,100.000000


## 2.2. Prepare dataset
#### Split the data into train and test sets with 0.2 test size.

In [5]:
X=df.drop([2],axis=1)
y=df[[2]]

In [6]:
X.head()

,0,1
0,10182.554999,-371.830691
1,-8493.323486,7009.446179
2,21322.088204,-390.558362
3,5473.925002,-1878.223941
4,-7422.540710,5291.351276


In [7]:
y.head()

,2
0,100.0
1,0.0
2,100.0
3,0.0
4,0.0


In [8]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=14)

In [13]:
print(X_train.shape[0],y_train.shape[0])

12000 12000


## 2.3. Modeling
#### Train a DecisionTreeClassifier on the data. Use GridSearchCV to tune the hyperparameters.

In [9]:
param_grid={
    'max_depth': [10, 20, 30],
    'min_samples_split': [3, 5, 10],
    'min_samples_leaf': [1, 2, 5]
}
dt=DecisionTreeClassifier()

In [10]:
grid_search=GridSearchCV(estimator=dt,param_grid=param_grid,cv=5,n_jobs=-1,verbose=2,scoring='accuracy')
grid_search.fit(X_train,y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'max_depth': [10, 20, 30],
                         'min_samples_leaf': [1, 2, 5],
                         'min_samples_split': [3, 5, 10]},
             scoring='accuracy', verbose=2)

#### Train the best model on the whole train set (do you need to?) and evaluate the model on the test set.



In [11]:
# Retrieve the best parameters from GridSearchCV
best_params = grid_search.best_params_
best_params


{'max_depth': 10, 'min_samples_leaf': 5, 'min_samples_split': 5}

In [14]:
# Initialize a new DecisionTreeClassifier with the best parameters
best_model = DecisionTreeClassifier(**best_params, random_state=42)

# Retrain the model on the full training set
best_model.fit(X_train, y_train)

# Evaluate on the test set
y_pred_test = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)

print("Test Accuracy after retraining:", test_accuracy)


Test Accuracy after retraining: 0.8386666666666667


#### Generate 1,200 subsets of the training set, each containing 100 randomly chosen instances. You can use ShuffleSplit.

In [15]:
# Initialize ShuffleSplit
n_subsets = 1200  # Number of subsets
subset_size = 100  # Size of each subset
n_instances = X_train.shape[0]

# ShuffleSplit with n_splits=1200
rs = ShuffleSplit(n_splits=n_subsets, train_size=subset_size)


#### Train one tree on each subset, using the best model you previously found. Evaluate the performance of the trees using the test set. Did you get lower or higher accuracy? Why?

In [18]:
# List to store accuracies of each tree
accuracies = []

# Loop through each subset
for train_idx, _ in rs.split(X_train, y_train):
    # Use .iloc to select rows by index for DataFrames
    X_subset = X_train.iloc[train_idx]  # Correct indexing for DataFrame
    y_subset = y_train.iloc[train_idx]  # Correct indexing for DataFrame

    best_model.fit(X_subset, y_subset)  # Train on the subset

    # Evaluate the model on the test set
    y_pred_test = best_model.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred_test)
    accuracies.append(accuracy)

# Calculate the average accuracy across all subsets
average_accuracy = np.mean(accuracies)

print(f"Average Test Accuracy from 1,200 trees: {average_accuracy}")

Average Test Accuracy from 1,200 trees: 0.794735


#### For each instance in the test set, predict its class using 1200 trees, and keep only the most frequent prediction. You can use mode from scipy.stats. Evaluate these predictions. Did you get lower or higher accuracy?

In [19]:
# List to store predictions from each tree
all_predictions = []

# Loop through each subset and train a tree on it
for train_idx, _ in rs.split(X_train, y_train):
    # Select the subset using indices
    X_subset = X_train.iloc[train_idx]  # Correct indexing for DataFrame
    y_subset = y_train.iloc[train_idx]  # Correct indexing for DataFrame

    # Train the model on the subset
    best_model.fit(X_subset, y_subset)

    # Get predictions on the test set
    y_pred_test = best_model.predict(X_test)

    # Store the predictions for later aggregation
    all_predictions.append(y_pred_test)

# Convert list of predictions to a numpy array for easier manipulation
all_predictions = np.array(all_predictions)

# For each instance in the test set, find the most frequent prediction
final_predictions = []

for i in range(len(X_test)):  # Loop over each instance in the test set
    # Get predictions for this instance from all 1,200 trees
    instance_predictions = all_predictions[:, i]

    # Get the most frequent prediction (mode) for this instance
    most_frequent_prediction = mode(instance_predictions)[0][0]

    final_predictions.append(most_frequent_prediction)

# Evaluate the final predictions
final_accuracy = accuracy_score(y_test, final_predictions)

print(f"Accuracy using mode of 1,200 trees: {final_accuracy}")

C:\Users\Ehtiram\AppData\Local\Temp\ipykernel_14188\75668937.py:30: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  most_frequent_prediction = mode(instance_predictions)[0][0]


Accuracy using mode of 1,200 trees: 0.854
